# Creating an IMDB Sentiment Analysis Web App
## Using PyTorch and SageMaker

## General Outline

1. Download / Retrieve the data.
2. Prepare / Process the data.
3. Upload the processed data to S3.
4. Build and Train RNN model.
5. Deploy the trained model for testing (using a batch transform job)
6. Use the model for testing.
7. Deploy the model again for the web app.

In [1]:
# Make sure that we use SageMaker 1.x
!pip install sagemaker==1.72.0

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


## Step 1: Downloading the data

In [2]:
# %mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

## Step 2: Preparing and Processing the data

In [3]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [4]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Combine the positive and negative reviews and shuffle the resulting records.

In [5]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [6]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Check and see an example of the data our model will be trained on.

In [7]:
print(train_X[100])
print(train_y[100])

Just came back from the first showing of Basic Instinct 2. I was going into it thinking it would be crappy based on preview critics and I was pleasantly surprised! If you liked the original Basic Instinct I think you will enjoy #2 just as much if not more. Great story that always keeps you wondering and thinking. The music is superb, reprising the original's theme. Don't go expecting Academy Award material, go to see it for enjoyment and fun. That's what movies are designed for -- escapism. I can't think of a better way to escape than to escape with Sharon Stone who is as sexy as she ever was. Am thinking about going to see it again this weekend. Go see it!
1


Processing steps:
- Remove html tags 
- Tokenize inputs

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

Apply `review_to_words` to one of the reviews in the training set.

In [9]:
review_to_words(train_X[100])

['came',
 'back',
 'first',
 'show',
 'basic',
 'instinct',
 '2',
 'go',
 'think',
 'would',
 'crappi',
 'base',
 'preview',
 'critic',
 'pleasantli',
 'surpris',
 'like',
 'origin',
 'basic',
 'instinct',
 'think',
 'enjoy',
 '2',
 'much',
 'great',
 'stori',
 'alway',
 'keep',
 'wonder',
 'think',
 'music',
 'superb',
 'repris',
 'origin',
 'theme',
 'go',
 'expect',
 'academi',
 'award',
 'materi',
 'go',
 'see',
 'enjoy',
 'fun',
 'movi',
 'design',
 'escap',
 'think',
 'better',
 'way',
 'escap',
 'escap',
 'sharon',
 'stone',
 'sexi',
 'ever',
 'think',
 'go',
 'see',
 'weekend',
 'go',
 'see']

In [10]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        # words_train = list(map(review_to_words, data_train))
        # words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [11]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


## Transform the data

- Represent each word as an integer 
    - Fix the size of our working vocabulary and we will only include the words that appear most frequently. 
    - Combine all of the infrequent words into a single category (labeled as `1`).
- For recurrent neural network, it will be convenient if the length of each review is the same. 
    - Pad short reviews with the category 'no word' (labeled as `0`).
    - Truncate long reviews.

### Create a word dictionary

Map words that appear in the reviews to integers. Fix the size of our vocabulary (including the 'no word' and 'infrequent' categories) to be `5000`.

In [12]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    all_data = [word for sentence in data for word in sentence]
    
    import collections
    # A dict storing the words that appear in the reviews along with how often they occur
    word_count = collections.Counter(all_data)
    
    #Sort the words found in `data`
    sorted_words = [item[0] for item in sorted(word_count.items(), reverse=True, key=lambda item: item[1])]
    
    word_dict = {}
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [13]:
word_dict = build_dict(train_X)

In [14]:
word_dict

{'movi': 2,
 'film': 3,
 'one': 4,
 'like': 5,
 'time': 6,
 'good': 7,
 'make': 8,
 'charact': 9,
 'get': 10,
 'see': 11,
 'watch': 12,
 'stori': 13,
 'even': 14,
 'would': 15,
 'realli': 16,
 'well': 17,
 'scene': 18,
 'look': 19,
 'show': 20,
 'much': 21,
 'end': 22,
 'peopl': 23,
 'bad': 24,
 'go': 25,
 'great': 26,
 'also': 27,
 'first': 28,
 'love': 29,
 'think': 30,
 'way': 31,
 'act': 32,
 'play': 33,
 'made': 34,
 'thing': 35,
 'could': 36,
 'know': 37,
 'say': 38,
 'seem': 39,
 'work': 40,
 'plot': 41,
 'two': 42,
 'actor': 43,
 'year': 44,
 'come': 45,
 'mani': 46,
 'seen': 47,
 'take': 48,
 'life': 49,
 'want': 50,
 'never': 51,
 'littl': 52,
 'best': 53,
 'tri': 54,
 'man': 55,
 'ever': 56,
 'give': 57,
 'better': 58,
 'still': 59,
 'perform': 60,
 'find': 61,
 'feel': 62,
 'part': 63,
 'back': 64,
 'use': 65,
 'someth': 66,
 'director': 67,
 'actual': 68,
 'interest': 69,
 'lot': 70,
 'real': 71,
 'old': 72,
 'cast': 73,
 'though': 74,
 'live': 75,
 'star': 76,
 'enjoy': 7

### Save `word_dict`
Save `word_dict` for future use.

In [16]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [17]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the reviews

Pad or truncate to a fixed length of `500`.

In [18]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [19]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

In [20]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
train_X[100]

array([ 149,    5,    3,   21,  111,  270,  135,  705,  272, 2183, 2116,
          3,    6,   19,  503,   26, 3095,  259,   61,    4, 3763, 1929,
          3,  500,  905, 1501,   44,   72,  162,   50,   37,  207,  921,
         93,  124,  909,  968, 2736, 4645,  365,  365,  322,  572, 1772,
        220,  419, 1876,    3,  933,  564,    8,    3,  234, 1331,   13,
        918, 3170,  322, 2116,  572,  748,    3,  564,   98,    1, 3202,
        751,    3, 1176,  172,  505,   20,   98,  435,   18,   92,  584,
       1176, 2417, 1054, 1723,    3,   97,  284,   31,    8,    2,   56,
         47,    2,    1,   40,    6,    1, 2230,    1,   25, 1871,  257,
        556, 2041,    5,   30,   21,   30,   15,   50,  224,    6, 1019,
        198,    2,   68,   16,    3, 2302, 1930,   57,   54,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [21]:
len(train_X[100])

500

## Step 3: Upload the data to S3
### Save the processed training dataset locally

In [22]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data to S3

In [23]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [24]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

## Step 4: Build and Train the PyTorch Model

Work on a small bit of the data to see how our training script is behaving.

In [26]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

In [27]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            optimizer.zero_grad()
            out = model.forward(batch_X)
            loss = loss_fn(out, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

In [28]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6933263897895813
Epoch: 2, BCELoss: 0.6845240354537964
Epoch: 3, BCELoss: 0.6772064089775085
Epoch: 4, BCELoss: 0.669486689567566
Epoch: 5, BCELoss: 0.6607130646705628


### Training the model

In [29]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    py_version="py3", # https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [30]:
estimator.fit({'training': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-12-25 05:53:52 Starting - Starting the training job...
2020-12-25 05:53:54 Starting - Launching requested ML instances.........
2020-12-25 05:55:24 Starting - Preparing the instances for training......
2020-12-25 05:56:48 Downloading - Downloading input data......
2020-12-25 05:57:47 Training - Downloading the training image...
2020-12-25 05:58:09 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-25 05:58:09,834 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-12-25 05:58:09,862 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-12-25 05:58:09,867 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-12-25 05:58:10,199 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-12-25 05:58:10,199 sa

## Step 5: Deploy the model for testing

In [31]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

## Step 6 - Use the model for testing

Once deployed, we can read in the test data and send it off to our deployed model to get some results. Once we collect all of the results we can determine how accurate our model is.

In [32]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [33]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [34]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.85228

### More testing

In [36]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

In [37]:
test_data = [convert_and_pad(word_dict, review_to_words(test_review))[0]]

In [38]:
predictor.predict(test_data)

array(0.51273274, dtype=float32)

Since the return value of our model is close to `1`, we can be certain that the review we submitted is positive.

### Delete the endpoint

In [39]:
estimator.delete_endpoint()

estimator.delete_endpoint() will be deprecated in SageMaker Python SDK v2. Please use the delete_endpoint() function on your predictor instead.


## Step 7 - Deploy the model again for the web app

### Deploying the model

In [41]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

# The default behaviour for a deployed PyTorch model is to assume that any input passed to the predictor is a `numpy` array. 
# In our case we want to send a string so we need to construct a simple wrapper around the `RealTimePredictor` class to accomodate simple strings.

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     py_version="py3", # https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

### Testing the model

In [42]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(float(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [43]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.866

In [45]:
predictor.predict(test_review)

b'1.0'

In [46]:
predictor.endpoint

'sagemaker-pytorch-2020-12-25-06-13-30-710'

## Selected Results

### Correct Sentiment

<img src="website/screenshot/LOR-Positive.png">

<img src="website/screenshot/StarWar-Negative.png">

### Incorrect Sentiment

<img src="website/screenshot/Thor-Positive.png">

<img src="website/screenshot/Twilight-Negative.png">

### Delete the endpoint

Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill.

In [47]:
predictor.delete_endpoint()